# Create a scoring script

In [3]:
# !pip install mlflow

In [1]:
import pickle
import pandas as pd

In [2]:
year = 2023
month = 3

input_file = f'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year:04d}-{month:02d}.parquet'
output_file = f'output-yellow-{year:04d}-{month:02d}.parquet'
model_file = 'web-service/model.bin'

categorical = ['PULocationID', 'DOLocationID']

In [3]:
def load_model(model_file):
    with open(model_file, 'rb') as f_in:
        dv, model = pickle.load(f_in)
    return dv, model

In [4]:
def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [5]:
def apply_model(input_file, model_file, output_file):
    df = read_data(input_file)
    dv, model = load_model(model_file)
    
    dicts = df[categorical].to_dict(orient='records')
    X_val = dv.transform(dicts)
    y_pred = model.predict(X_val)


    df['ride_id'] = f'{year}/{month}_' + df.index.astype('str')
    df["y_pred"] = y_pred
    output_columns = ["ride_id", "y_pred"]
    df_result = df[output_columns]
    
    df_result.to_parquet(
        output_file,
        engine='pyarrow',
        compression=None,
        index=False
        )

In [6]:
apply_model(input_file=input_file, model_file=model_file, output_file=output_file)

/home/codespace/anaconda3/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator DictVectorizer from version 1.5.0 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/codespace/anaconda3/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator LinearRegression from version 1.5.0 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
